In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
# 1. Generate Dummy Student Data
np.random.seed(0)
N = 60
data = pd.DataFrame({
    'hours_studied': np.random.randint(1, 10, N),          # 1 to 9 hours/day
    'attendance_pct': np.random.randint(50, 101, N),       # 50% to 100% attendance
    'assignments_done': np.random.randint(5, 21, N)        # 5 to 20 assignments
})

In [ ]:
# Generate exam score (continuous)
data['exam_score'] = (
    data['hours_studied'] * 8 +
    data['attendance_pct'] * 2 +
    data['assignments_done'] * 3 +
    np.random.normal(0, 15, N)  # Noise
)

In [ ]:

# Discretize target: Low=0, Medium=1, High=2
bins = [0, 500, 800, np.inf]
labels = [0, 1, 2]
data['score_class'] = pd.cut(data['exam_score'], bins=bins, labels=labels).astype(int)

X = data[['hours_studied', 'attendance_pct', 'assignments_done']]
y = data['score_class']
n_classes = len(y.unique())

In [ ]:
# 2. Train-Test Split
# =============================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Convert to NumPy
X_train_np = X_train.values
y_train_np = y_train.values
X_test_np = X_test.values
y_test_np = y_test.values

In [ ]:
# 3. Logistic Regression (One-vs-Rest)
# =============================
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def train_one_vs_rest(X, y, n_classes, lr=1e-4, n_iter=5000):
    n_samples, n_features = X.shape
    weights = []
    for k in range(n_classes):
        yk = (y == k).astype(np.float64)
        w = np.zeros(n_features)
        b = 0.0
        for _ in range(n_iter):
            logits = np.dot(X, w) + b
            y_pred = sigmoid(logits)
            dw = np.dot(X.T, (y_pred - yk)) / n_samples
            db = np.sum(y_pred - yk) / n_samples
            w -= lr * dw
            b -= lr * db
        weights.append({'w': w, 'b': b})
    return weights

def predict(X, weights):
    n_samples = X.shape[0]
    n_classes = len(weights)
    probs = np.zeros((n_samples, n_classes))
    for k in range(n_classes):
        w = weights[k]['w']
        b = weights[k]['b']
        logits = np.dot(X, w) + b
        probs[:, k] = sigmoid(logits)
    return np.argmax(probs, axis=1)


In [ ]:
# 4. Train & Evaluate
# =============================
weights = train_one_vs_rest(X_train_np, y_train_np, n_classes)
y_pred = predict(X_test_np, weights)

print("Accuracy:", accuracy_score(y_test_np, y_pred))
print("\nClassification Report:\n", classification_report(y_test_np, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test_np, y_pred))

Accuracy: 1.0

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        12

    accuracy                           1.00        12
   macro avg       1.00      1.00      1.00        12
weighted avg       1.00      1.00      1.00        12


Confusion Matrix:
 [[12]]


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


In [ ]:
# 5. Sample Predictions
# =============================
results = X_test.copy()
results['true_score'] = data.loc[X_test.index, 'exam_score']
results['true_class'] = y_test
results['pred_class'] = y_pred
print(results.head())

    hours_studied  attendance_pct  assignments_done  true_score  true_class  \
22              4              85                 5  245.013384           0   
28              4              99                15  289.208780           0   
42              8              79                18  264.451259           0   
20              9              65                 8  254.844130           0   
14              7              97                15  275.942725           0   

    pred_class  
22           0  
28           0  
42           0  
20           0  
14           0  


In [ ]:
# 6. Class Distribution
# =============================
print("\nClass distribution:")
print(data['score_class'].value_counts())


Class distribution:
score_class
0    60
Name: count, dtype: int64
